# Models

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lukeconibear/intro_ml/blob/main/docs/04_models.ipynb)

In [ ]:
# if you're using colab, then install the required modules
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install --quiet keras-tuner --upgrade

```{note}
If you’re in COLAB or have a local CUDA GPU, you can follow along with the more computationally intensive training in this lesson.

For those in COLAB, ensure the session is using a GPU by going to: Runtime > Change runtime type > Hardware accelerator = GPU.
```

## Hyperparameter tuning

Tune the [hyperparameters](hyperparameters) to find the best model.

### TensorFlow (Keras)

[KerasTuner](https://keras.io/guides/keras_tuner/getting_started/) is a library the helps you pick the best hyperparameters for your model.

#### Other options

- [TensorFlow Model Optimisation Toolkit](https://www.tensorflow.org/model_optimization)
    - A suite of tools for optimising machine learning models.
- [AutoKeras](https://autokeras.com/)
    - An AutoML system to automate the building of the machine learning model.

In [ ]:
import os

import keras_tuner
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))

Build the model with the `hyperparameters` option.

- The number of units.
- The activation function to use
- Whether to use dropout.
- The optimiser learning rate.

In [ ]:
def build_model(hyperparameters):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    model.add(
        tf.keras.layers.Dense(
            units=hyperparameters.Int("units", min_value=32, max_value=512, step=32),
            activation=hyperparameters.Choice("activation", ["relu", "tanh"]),
        )
    )
    if hyperparameters.Boolean("dropout"):
        model.add(tf.keras.layers.Dropout(rate=0.25))
    model.add(tf.keras.layers.Dense(10, activation="softmax"))
    learning_rate = hyperparameters.Float(
        "lr", min_value=1e-4, max_value=1e-2, sampling="log"
    )
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

First, prepare a directory to store all the hyperparameters:

In [ ]:
hyperparameters_path = f"{os.getcwd()}/models/hyperparameters"
if not os.path.exists(hyperparameters_path):
    os.makedirs(hyperparameters_path)

Now, initialise the tuner.

This uses Random Search, though you could also use [other methods](https://www.tensorflow.org/tutorials/keras/keras_tuner#instantiate_the_tuner_and_perform_hypertuning).

In [ ]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,  # the model building function
    objective="val_accuracy",  # the objective to optimise
    max_trials=3,  # the number of trials for the search
    executions_per_trial=2,  # the number of models to build and fit per trial
    overwrite=True,  # whether to overwrite previous results
    directory=hyperparameters_path,  # the path for the hyperparameter results
    project_name="hp_example",
)

View the search space:

In [ ]:
tuner.search_space_summary()

Load in the MNIST dataset for this example:

In [ ]:
(x, y), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x[:-10000]
x_val = x[-10000:]
y_train = y[:-10000]
y_val = y[-10000:]

x_train = np.expand_dims(x_train, -1).astype("float32") / 255.0
x_val = np.expand_dims(x_val, -1).astype("float32") / 255.0
x_test = np.expand_dims(x_test, -1).astype("float32") / 255.0

num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

Start the search:

In [ ]:
if IN_COLAB:
    tuner.search(
        x_train, y_train, epochs=2, validation_data=(x_val, y_val), verbose=False
    )

View the results.

You can also visualise them using [TensorBoard](https://keras.io/guides/keras_tuner/visualize_tuning/).results

In [ ]:
if IN_COLAB:
    tuner.results_summary()

Select the best model:

In [ ]:
if IN_COLAB:
    best_model = tuner.get_best_models()[0]

Show the best model's summary.

Note, the `Sequential` model will need to be built first with a input shape.

In [ ]:
if IN_COLAB:
    best_model.build(input_shape=(None, 28, 28))
    best_model.summary()

### PyTorch (Lightning)

In [ ]:
import pytorch_lightning as pl

## [Transfer learning](https://youtu.be/yofjFQddwHE)

Transfer learning is where a model that has been pre-trained on a problem is transferred to another similar problem. [For example](https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/), if a model learnt to classify images it can be transferred to other image classification problems.

This works because the pre-trained model has _extracted features_ from the similar task (i.e., lower-level: lines, curves, etc. and higher-level: eyes, ears, etc.).

Commonly, this is done via:

- Instantiate a pre-trained model and load pre-trained weights into it.
- Extract layers and freeze them (i.e., `trainable = False`) to retain their information (e.g., for lower level features).
- Add new trainable layers on top of these frozen layers.
- Train the new layers on your dataset.
- Optional: Unfreeze the frozen layers and train these on the dataset with a very low learning rate (i.e., [fine tuning](https://www.tensorflow.org/tutorials/images/transfer_learning#fine_tuning)).
    - This is done when the training dataset is larger and very similar to the original pre-trained dataset.

This is useful when:

- You have a small dataset.
- You want to take advantage of huge models without the costs of training them.

There are range of pre-trained models available in [Keras Applications](https://keras.io/api/applications/).

### Example: Transfer learning

#### [TensorFlow (Keras)](https://www.tensorflow.org/tutorials/images/transfer_learning)

Download and split the data:

In [ ]:
if IN_COLAB:
    # tfds.disable_progress_bar()

    ds_train, ds_val, ds_test = tfds.load(
        "cats_vs_dogs",
        # Reserve 10% for validation and 10% for test
        split=["train[:40%]", "train[40%:50%]", "train[50%:60%]"],
        as_supervised=True,  # Include labels
    )

    print("Number of training samples: %d" % tf.data.experimental.cardinality(ds_train))
    print("Number of validation samples: %d" % tf.data.experimental.cardinality(ds_val))
    print("Number of test samples: %d" % tf.data.experimental.cardinality(ds_test))

View a few of the training samples:

In [ ]:
if IN_COLAB:
    labels = {0: "Cat", 1: "Dog"}

    plt.figure(figsize=(10, 10))
    for i, (image, label) in enumerate(ds_train.take(9)):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(image)
        plt.title(labels[int(label)])
        plt.axis("off")

First, preprocess the data.

This is to standardise the image sizes and normalise the pixel values.

In [ ]:
if IN_COLAB:
    IMAGE_SIZE = (150, 150)

    ds_train = ds_train.map(lambda x, y: (tf.image.resize(x, IMAGE_SIZE), y))
    ds_val = ds_val.map(lambda x, y: (tf.image.resize(x, IMAGE_SIZE), y))
    ds_test = ds_test.map(lambda x, y: (tf.image.resize(x, IMAGE_SIZE), y))

Now, we can [cache](cache_tf), [batch](batch_tf), and [prefetch](prefetch_tf) the data:

In [ ]:
if IN_COLAB:
    BATCH_SIZE = 32
    AUTOTUNE = tf.data.AUTOTUNE

    ds_train = ds_train.cache().batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
    ds_val = ds_val.cache().batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
    ds_test = ds_test.cache().batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

Apply some [data augmentation](data_augmentation):

In [ ]:
data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.1),
    ]
)

Now, build the model.

Steps:

- First, load the pre-trained model as the base.
    - This example uses the Xception model (other options are [here](https://keras.io/api/applications/)).
    - The weights were pre-trained on ImageNet (a large image dataset).
    - The top layer from the Xception classifier is not included (i.e., the one that predicts the ImageNet categories).
- Freeze the layers from the base model.
- Add new layers on top.

```{note}

[BatchNormalisation](https://www.tensorflow.org/tutorials/images/transfer_learning#important_note_about_batchnormalization_layers) layers should be kept in inference mode (i.e., the base model should keep `training = False`) when the base model is unfrozen for fine-tuning.

```

In [ ]:
base_model = tf.keras.applications.Xception(
    weights="imagenet",  # use weights from ImageNet
    input_shape=(150, 150, 3),
    include_top=False,  # don't include Xceptions classifier for ImageNet
)
base_model.trainable = False  # freeze the pre-trained model

inputs = tf.keras.Input(shape=(150, 150, 3))  # create new model on top
x = data_augmentation(inputs)  # apply data augmentation

scale_layer = tf.keras.layers.Rescaling(
    scale=1.0 / 127.5, offset=-1
)  # rescale from [0, 255] to [-1.0, 1.0]
x = scale_layer(x)

x = base_model(
    x, training=False
)  # keep batchnorm layers in infernece mode (see note above)
x = tf.keras.layers.GlobalAveragePooling2D()(x)  # convert 2D locations to vector
x = tf.keras.layers.Dropout(0.2)(x)  # for regularisation

outputs = tf.keras.layers.Dense(1)(x)  # prediction layer

model = tf.keras.Model(inputs, outputs)

model.summary()

1% of the parameters are trainable, with 99% from the pre-trained model.

That will save a lot of computation.

Now, train the top layers only:

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.BinaryAccuracy(name="accuracy")],
)

In [ ]:
if IN_COLAB:
    # just 1 epoch for the demonstration, would increase this to say 20 normally
    NUM_EPOCHS = 1
    model.fit(ds_train, epochs=NUM_EPOCHS, validation_data=ds_val)

Now you could perform [fine tuning](https://www.tensorflow.org/tutorials/images/transfer_learning#fine_tuning).

This is where you unfreeze the frozen layers and train these on the dataset with a very low learning rate.

#### [TensorFlow Hub](https://www.tensorflow.org/tutorials/images/transfer_learning_with_hub)

There are many [pre-trained models](https://tfhub.dev/) on [TensorFlow Hub](https://www.tensorflow.org/hub/overview) for use in transfer learning.

Download the data:

In [ ]:
data_root = tf.keras.utils.get_file(
    "flower_photos",
    "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz",
    untar=True,
)

In [ ]:
BATCH_SIZE = 32
IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224

ds_train = tf.keras.utils.image_dataset_from_directory(
    str(data_root),
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    str(data_root),
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
)

In [ ]:
class_names = np.array(ds_train.class_names)
num_classes = len(class_names)
class_names

Pre-process the data:

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1.0 / 255)
ds_train = ds_train.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

Create the data pipeline:

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
ds_train = ds_train.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

Download the headless pre-trained model (i.e., without the top layer):

In [ ]:
mobilenet_v2 = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
inception_v3 = "https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4"

feature_extractor_model = (
    mobilenet_v2  # change to inception_v2, or choose a different one
)

The download location defaults to a [local temporary directory](https://www.tensorflow.org/hub/caching).

To change this, set the following:

```python
import os
os.environ["TFHUB_CACHE_DIR"] = "/nobackup/username/tf_hub_modules"
```

Create the headless pre-trained _layer_ by setting trainable to False:

In [ ]:
import tensorflow_hub as hub

In [ ]:
feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3), trainable=False
)

Create the new model with the pre-trained layer as the base and a new classification layer on top:

In [ ]:
model = tf.keras.Sequential(
    [feature_extractor_layer, tf.keras.layers.Dense(num_classes)]
)

model.summary()

Compile the new model:

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

Train the new model:

In [ ]:
if IN_COLAB:
    NUM_EPOCHS = 3
    history = model.fit(ds_train, validation_data=val_ds, epochs=NUM_EPOCHS)

#### [PyTorch (Lightning)](https://pytorch-lightning.readthedocs.io/en/stable/advanced/transfer_learning.html)

In [15]:
import torch
from pytorch_lightning import LightningModule
import torchvision.models as models

In [16]:
class ImagenetTransferLearning(LightningModule):
    def __init__(self):
        super().__init__()

        # init a pretrained resnet
        backbone = models.resnet50(pretrained=True)
        
        # extract some setup details
        num_filters = backbone.fc.in_features
        layers = list(backbone.children())[:-1]
        self.feature_extractor = nn.Sequential(*layers)

        # use the pretrained model to classify cifar-10 (10 image classes)
        num_target_classes = 10
        self.classifier = nn.Linear(num_filters, num_target_classes)

    def forward(self, x):
        self.feature_extractor.eval()
        with torch.no_grad():
            representations = self.feature_extractor(x).flatten(1)
        x = self.classifier(representations)
        # ... the rest of your LightningModule ...

#### [PyTorch Lightning Bolts](https://lightning-bolts.readthedocs.io/en/latest/introduction_guide.html#for-pretrained-models)

In [17]:
from pl_bolts.models.autoencoders import VAE

In [21]:
model = VAE(input_height=32, pretrained='imagenet2012')

In [22]:
encoder = model.encoder
encoder.eval()

ResNetEncoder(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): EncoderBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): EncoderBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inpla

#### [PyTorch Lightning-Flash](https://lightning-flash.readthedocs.io/en/latest/general/finetuning.html)

In [7]:
import os

import flash
import torch
from flash.core.classification import LabelsOutput
from flash.core.data.utils import download_data
from flash.image import ImageClassificationData, ImageClassifier
from pytorch_lightning import seed_everything

In [2]:
seed_everything(42)

Global seed set to 42


42

Download and organize the data:

In [3]:
data_path = f"{os.getcwd()}/data"

In [12]:
download_data("https://pl-flash-data.s3.amazonaws.com/hymenoptera_data.zip", f"{data_path}/")

datamodule = ImageClassificationData.from_folders(
    train_folder=f"{data_path}/hymenoptera_data/train/",
    val_folder=f"{data_path}/hymenoptera_data/val/",
    test_folder=f"{data_path}/hymenoptera_data/test/",
    batch_size=1,
)

/home/earlacoa/miniconda3/envs/swd8_intro_ml/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pl-flash-data.s3.amazonaws.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Build the model using desired Task (e.g., [ResNet18](https://arxiv.org/pdf/1512.03385.pdf)):

In [5]:
model = ImageClassifier(backbone="resnet18", labels=datamodule.labels)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/earlacoa/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

/home/earlacoa/miniconda3/envs/swd8_intro_ml/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:244: UserWarning: Attribute 'adapter' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['adapter'])`.
  rank_zero_warn(
/home/earlacoa/miniconda3/envs/swd8_intro_ml/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:244: UserWarning: Attribute 'backbone' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['backbone'])`.
  rank_zero_warn(
/home/earlacoa/miniconda3/envs/swd8_intro_ml/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:244: UserWarning: Attribute 'head' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['head'])`.
  rank_zero_warn(


Create the trainer (run one epoch for demo):

In [8]:
trainer = flash.Trainer(max_epochs=1, gpus=torch.cuda.device_count())

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Finetune the model.

`strategy="freeze"` keeps the pre-trained model (backbone) frozen throughout.

In [9]:
if IN_COLAB:
    trainer.finetune(model, datamodule=datamodule, strategy="freeze")


  | Name          | Type           | Params
-------------------------------------------------
0 | train_metrics | ModuleDict     | 0     
1 | val_metrics   | ModuleDict     | 0     
2 | test_metrics  | ModuleDict     | 0     
3 | adapter       | DefaultAdapter | 11.2 M
-------------------------------------------------
10.6 K    Trainable params
11.2 M    Non-trainable params
11.2 M    Total params
44.710    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/earlacoa/miniconda3/envs/swd8_intro_ml/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/earlacoa/miniconda3/envs/swd8_intro_ml/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:72: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/home/earlacoa/miniconda3/envs/swd8_intro_ml/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing t

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Save the model:

In [10]:
model_path = f"{os.getcwd()}/models"

In [11]:
if IN_COLAB:
    trainer.save_checkpoint(f"{model_path}/image_classification_model.pt")

## [Callbacks](https://keras.io/api/callbacks/)

Callbacks are objects that get called by the model at different points during training, in particular after each batch or epoch.

For example, they could be used to:

- Save a model version at regularly intervals or once attained a metric threshold (i.e., checkpointing).
- Monitor and profile the training progress (i.e., TensorBoard)
- Change the learning rate when training plateaus.
- Fine tuning when the training plateaus.

### Checkpoints

You can save a model (/ model weights) at custom checkpoints e.g., the latest best model in terms of accuracy per epoch.

The `save_best_only` option is useful as after training only the best model will have been saved.

#### [TensorFlow (Keras)](https://www.tensorflow.org/guide/checkpoint)

In [ ]:
path_models = f"{os.getcwd()}/models"

In [ ]:
callback_tf_model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=path_models + "/model_{epoch}",
    save_freq="epoch",  # save a model version at the end of each epoch
    save_best_only=True,  # only save a model if val_accuracy improved
    monitor="val_accuracy",
    # save_weights_only=True
)

#### PyTorch (Lightning)

Checkpointing enabled by default in PyTorch Lightning.

Model checkpoints are saved per epoch to `lightning_logs/version_X/checkpoints`.

### Fault tolerance

For longer or distributed training, it's helpful to save the model at regular intervals in case it crashes during training.

#### [TensorFlow (Keras)](https://www.tensorflow.org/tutorials/distribute/multi_worker_with_keras#fault_tolerance)

This is done via [BackupAndRestore](https://www.tensorflow.org/tutorials/distribute/multi_worker_with_keras#checkpoint_saving_and_restoring) (previously was done using ModelCheckpoint).

In [ ]:
callback_tf_model_backup = tf.keras.callbacks.BackupAndRestore(
    backup_dir=f"{path_models}/backup"
)

#### [PyTorch (Lightning)](https://pytorch-lightning.readthedocs.io/en/stable/advanced/fault_tolerant_training.html)

By default, if `Trainer.fit()` fails, then it can be restarted automatically from the _beginning_ of the epoch it failed on.

With Fault Tolerant Training, when `Trainer.fit()` fails in the middle of an epoch during training or validation, Lightning will restart exactly where it failed, and everything will be restored.

Enabled via:

```bash
PL_FAULT_TOLERANT_TRAINING=1 python script.py
```

### Logging and Profiling

[Tensorboard](https://www.tensorflow.org/tensorboard) is a browser-based application that provides live plots of loss and metrics for training and evaluation.

#### [TensorFlow (Keras)](https://www.tensorflow.org/guide/profiler)

In [ ]:
callback_tf_tensorboard_with_profiling = tf.keras.callbacks.TensorBoard(
    log_dir=f"{path_models}/logs",
    profile_batch=(1, 5),  # profile batches 1 to 5
    update_freq="epoch",
)

In [ ]:
# remove previous logs
!rm -r logs

View them with:

```bash
tensorboard --logdir=/full_path_to_your_logs
```

Also, in-line in [Jupyter Notebooks / Google Colab](https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks):

```python
from tensorboard import notebook
notebook.list() # list open tensorboard instances
notebook.display(port=6006, height=1000) # open tensorboard

# or

%load_ext tensorboard
%tensorboard --logdir logs
```

#### PyTorch (Lightning)

##### [Logging](https://pytorch-lightning.readthedocs.io/en/stable/extensions/logging.html)

PyTorch Lightning uses TensorBoard for loggin by default.  

You can view the logs using:

```python
%load_ext tensorboard
%tensorboard --logdir lightning_logs/
```

##### Profiling

You can profile (time and memory) with [PyTorch](https://pytorch.org/tutorials/recipes/recipes/profiler_recipe.html) using the `torch.autograd.profiler` context manager:

In [24]:
import torch
import torchvision.models as models
import torch.autograd.profiler as profiler

In [25]:
model = models.resnet18()
inputs = torch.randn(5, 3, 244, 244)

In [26]:
with profiler.profile() as prof:
    with profiler.record_function("model_inference"):
        model(inputs)

This can then help you find bottlenecks in the code:

In [28]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=5))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  model_inference         4.70%       7.858ms        99.99%     167.127ms     167.127ms             1  
                     aten::conv2d         0.05%      91.000us        74.49%     124.503ms       6.225ms            20  
                aten::convolution         0.19%     311.000us        74.43%     124.412ms       6.221ms            20  
               aten::_convolution         0.11%     189.000us        74.25%     124.101ms       6.205ms            20  
         aten::mkldnn_convolution        73.99%     123.673ms        74.13%     123.912ms       6.196ms            20  
---------------------------------  -----

In [PyTorch Lightning](https://pytorch-lightning.readthedocs.io/en/stable/advanced/profiler.html), for a simple profiler over method calls and time spent calling them:

```python
Trainer(profiler=True)
```

For more control, use:

```python
from pytorch_lightning.profiler import AdvancedProfiler

Trainer(profiler=AdvancedProfiler())
```

### Early stopping

Stop training when a monitored metric has stopped improving (i.e., early stopping).

#### [TensorFlow (Keras)](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping)

In [ ]:
callback_tf_early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",  # quantity to be monitored
    patience=2,  # "no longer improving" also means "for at least 2 epochs"
)

#### [PyTorch (Lightning)](https://pytorch-lightning.readthedocs.io/en/stable/common/early_stopping.html#)

In [ ]:
callback_torch_early_stopping = pl.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=2,
)

### Learning rate decay

Reduce learning rate when a metric has stopped improving (i.e., reduce the learning rate on plateau).

#### [TensorFlow (Keras)](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ReduceLROnPlateau)

In [ ]:
callback_tf_learning_rate_decay = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_accuracy",
    patience=5,
    factor=0.2,  # factor by which the learning rate will be reduced: new_lr = lr * factor
    min_lr=0.001,  # lower bound on the learning rate
)

### Example: Callbacks

#### TensorFlow (Keras)

In [ ]:
callbacks_tf = [
    callback_tf_model_checkpoint,
    callback_tf_model_backup,
    callback_tf_tensorboard_with_profiling,
    callback_tf_early_stopping,
    callback_tf_learning_rate_decay,
]

In [ ]:
if IN_COLAB:
    epochs = 5  # just 1 for the demonstration, would increase this to say 20 normally
    model.fit(ds_train, epochs=epochs, validation_data=ds_val, callbacks=callbacks_tf)

View the results in TensorBoard:

In [ ]:
# from tensorboard import notebook
# notebook.list()
# notebook.display(port=6006, height=1000)

In [ ]:
if IN_COLAB:
    %load_ext tensorboard
    %tensorboard --logdir /content/models/logs

#### PyTorch (Lightning)

In [ ]:
# callbacks_torch = [
#     callback_torch_early_stopping,
# ]

In [ ]:
# Trainer(callbacks=callbacks_torch)

## Compiling

Compile any function in TensorFlow by wrapping it in the [`@tf.function`](https://www.tensorflow.org/api_docs/python/tf/function) decorator.

This convert it from eager execution to a static graph.



`torch.jit`

## Exercises

```{admonition} Exercise 1

...

```

## {ref}`Solutions <models>`

## Key Points

```{important}

- [x] _Tune the models hyperparamaters for the best fit._
- [x] _Use transfer learning to save computation on similar problems._

```

## Further information

### Good practices

- See if there is a model architecture (and parameters) that already addresses the task.
- Consider the tradeoff between model complexity and size.
    - For high accuracy, maybe need a large and complex model.
    - For less precision, smaller models use less disk space, memory, and are faster.
- Best practices for [PyTorch Lightning model training](https://pytorch-lightning.readthedocs.io/en/stable/guides/speed.html)

### Other options

There are many other options for hyperparameters tuning, including:

- [Hyperopt](http://hyperopt.github.io/hyperopt/)
    - A general purpose optimisation library.
- [scikit-optimise](https://scikit-optimize.github.io/stable/)
    - A general purpose optimisation library.
- [TPOT](http://epistasislab.github.io/tpot/)
    - Automated optimisation library using genetic programming.
 
### Resources

#### General

- [Model Zoo](https://modelzoo.co/)
- [Papers with code - Models](https://paperswithcode.com/methods)
- [HuggingFace - Models](https://huggingface.co/models)

#### TensorFlow

- [TensorFlow Model Garden](https://github.com/tensorflow/models/tree/master/official) for model source code.

#### PyTorch

- [PyTorch Hub](https://pytorch.org/docs/stable/hub.html) for pre-retrained models.
- [Torch Vision Models](https://pytorch.org/vision/stable/models.html)
- [Torch Text Models](https://pytorch.org/text/stable/models.html)
- [Torch Audio Models](https://pytorch.org/audio/stable/models.html)
- [TIMM (pyTorch IMage Models)](https://rwightman.github.io/pytorch-image-models/)